In [2]:
import numpy as np
import pandas as pd

# Preprocessing

## Step 1


You don't have to run **Step 1** if there is the *../data/tmp/df_time.csv* file exists.

---

In [2]:
COLUMNS = ['station','date','feature', 'value', 'measurement','quality', 'source', 'hour']

In [3]:
df = pd.read_csv('../data/2014.csv', header=None, names=COLUMNS)

In [4]:
df = df.append(pd.read_csv('../data/2015.csv', header=None, names=COLUMNS))
df = df.append(pd.read_csv('../data/2016.csv', header=None, names=COLUMNS))
df = df.append(pd.read_csv('../data/2017.csv', header=None, names=COLUMNS))

In [5]:
df.head()

,station,date,feature,value,measurement,quality,source,hour
0,ASN00015643,20140101,TMAX,424,NaN,NaN,a,NaN
1,ASN00015643,20140101,TMIN,227,NaN,NaN,a,NaN
2,ASN00015643,20140101,PRCP,0,NaN,NaN,a,NaN
3,US1MNCV0008,20140101,PRCP,0,NaN,NaN,N,NaN
4,US1MNCV0008,20140101,SNOW,0,NaN,NaN,N,NaN


In [6]:
#Only selecting features who are available in Test Set + Feature to be predicted
selected_features = ['TMIN']

In [7]:
df = df[df['feature'].isin(selected_features)]

In [8]:
df_pivot = df.pivot_table(index=['station','date'], columns='feature', values='value', aggfunc=np.min)

In [19]:
df_time = df_pivot['TMIN']
df_time = df_time.reset_index()
len(df_time)

# Export df_time, so we don't have to load all of that data all the time.
# df_time.to_csv('../data/tmp/df_time.csv')

## Step 2

Make sure you have loaded `df_time` either in **Step 1** or you subsequently load it from a saved file.

---

In [23]:
df_time_path = '../data/tmp/df_time.csv'
df_time = pd.read_csv(df_time_path)
df_time = df_time.drop(['Unnamed: 0'], axis=1)

In [4]:
#Include Location Information?
df_stations = pd.read_csv('../data/ghcnd-stations.csv', header=None, names=['station','lat', 'long', 'elev'], sep=';')
df_stations = df_stations.set_index('station')
stations = df_time.station.unique()

In [40]:
def derive_all_nth_day_features(input_data, feature, N, export_file):
    '''
        Generates a new dataframe with <feature>_1 .. <feature>_N columns
        which represent the feature values of the previous N days.
    
        @param input_data:  Pandas dataframe with columns 
                            'station', 'date' and <feature>.
                            Needs to be ordered by station and date.
        @param feature:     Name of the feature column that should be used.
        @param N:           How many feature columns should be generated.
        @param_export_file  The open csv. file to which should be exported.
    '''
    
    # avoid side effects on input dataframe
    data = input_data.copy()
    
    # add empty columns
    col_name_for = lambda n: "{}_{}".format(feature, n)
    for i in range(1, N+1):
        data[col_name_for(i)] = [None]*len(data)
        
    # create empty dataframe for export
    SHOULD_WRITE_HEADERS = True
    COLUMN_NAMES = list(df_time.columns.values)
    export_data_frame = pd.DataFrame(columns=COLUMN_NAMES)

    # iterate over all stations and write compute the new columns
    for i, station in enumerate(stations):
        rows_for_station = data[data['station']==station]
        first_index = rows_for_station.index[0]
        
        # add verbose output to see whether stuff is still running
        print (''*20, '\r', end='')
        print (i+1, '/', len(stations), '::', station, '\r', end='')
        
        for row_index in range(first_index, len(rows_for_station)):
            for i in range(1, N+1):
                if (row_index - i >= first_index):
                    data.at[row_index, col_name_for(i)] = data.loc[row_index-i,feature]
        
        # append to export frame
        export_data_frame = export_data_frame.append( data.iloc[first_index:first_index+len(rows_for_station)])
        
        # Note: writing data takes very long, so only write every 2000 stations
        SAVE_EVERY_N_STATIONS = 250
        if (i%SAVE_EVERY_N_STATIONS == 0):
            print(i, 'w', end='')
            
            export_data_frame.to_csv(f, header=SHOULD_WRITE_HEADERS)
            
            # only write headers in the first run
            SHOULD_WRITE_HEADERS = False

            # empty the export_data_frame again
            export_data_frame = pd.DataFrame(columns=COLUMN_NAMES)
        
    print('')
    return data

In [41]:
N = 20
tmp_file_path = '../data/tmp/export_features_2014_to_2017_{}.csv'.format(N)
file_path = '../data/export_features_2014_to_2017_{}.csv'.format(N)
with open(tmp_file_path, 'a') as f:

    df_train = derive_all_nth_day_features(df_time, 'TMIN', N, f)
    f.close()
    
    # write entire set
    df_train.to_csv(file_path)
    

df_train.head()

KeyboardInterrupt: 